# Examples of usage

Here are simple examples of using stweet

In [ ]:
import stweet as st
import arrow

## 1. Scrap tweets

### Example 1.1.

Scrap tweets with `#covid19` between `'2020-05-11T20:00:00.000+01:00'` and `'2020-05-11T20:01:00.000+01:00'`.
Collect all tweets in memory and save to CSV file.

In [ ]:
task = st.SearchTweetsTask(
    '#covid19',
    since=arrow.get('2020-05-11T20:00:00.000+01:00'),
    until=arrow.get('2020-05-11T20:01:00.000+01:00')
)
tweets_collector = st.CollectorTweetOutput()
result = st.TweetSearchRunner(task, [tweets_collector, st.CsvTweetOutput('covid19_tweets.csv')]).run()
print(f'scrapping task result: {result}')

In [ ]:
print('first tweet content:')
print(tweets_collector.get_scrapped_tweets()[0].full_text)

In [ ]:
print(f'first tweet time: {tweets_collector.get_scrapped_tweets()[0].created_at}')

Scrapped time have different hour than was input in task, but everything is ok because scrapped tweet has no time offset.

### Example 1.2.

Scrap first 70 tweets with exact phrase `crypto is awesome` and save them in memory and in JSON lines file.

In [ ]:
task = st.SearchTweetsTask(
    exact_words='crypto is awesome',
    tweets_limit=70
)
tweets_collector = st.CollectorTweetOutput()
result = st.TweetSearchRunner(task, [tweets_collector, st.JsonLineFileTweetOutput('covid19_tweets.jl')]).run()
print(f'scrapping task result: {result}')

In [ ]:
print('first tweet content:')
print(tweets_collector.get_scrapped_tweets()[0].full_text)

In [ ]:
print('last tweet content:')
print(tweets_collector.get_scrapped_tweets()[-1].full_text)

### Example 1.3.

Scrap first 1000 tweets with any of hashtags: `#covid19` or `#bbc` using proxy. Save it in memory.

In [ ]:
task = st.SearchTweetsTask(
    any_word='#covid19 #bbc',
    tweets_limit=1000
)
web_client = st.RequestsWebClient(
    proxy=st.RequestsWebClientProxyConfig(http_proxy='', https_proxy='')
)
tweets_collector = st.CollectorTweetOutput()
result = st.TweetSearchRunner(task, [tweets_collector], web_client=web_client).run()
print(f'scrapping task result: {result}')

Twitter index hashtags without case sensitive. There is a need to create simple function to check that tweet contains hashtag.

In [ ]:
def tweet_cointain_hashtag(tweet: st.Tweet, hashtag: str) -> bool:
    return hashtag.lower() in tweet.full_text.lower()

In [ ]:
tweets = tweets_collector.get_scrapped_tweets()

print('both count:', len([
    it for it in tweets 
    if tweet_cointain_hashtag(it, '#bbc') and tweet_cointain_hashtag(it, '#covid19')
]))
print('only #covid19 count:', len([it for it in tweets if tweet_cointain_hashtag(it, '#covid19')]))
print('only #bbc count:', len([it for it in tweets if tweet_cointain_hashtag(it, '#bbc')]))

### Example 1.4.

Scrap tweets by ids and save in memory, check that all are existing.

In [ ]:
task = st.TweetsByIdsTask(['1337071849772093442', '1337067073051238400'])
tweets_collector = st.CollectorTweetOutput()

result = st.TweetsByIdsRunner(task, [tweets_collector]).run()
scrapped_tweets_ids = [it.id_str for it in tweets_collector.get_scrapped_tweets()]

print('tweet ids not scrapped:', result.tweet_ids_not_scrapped)
print('scrapped tweets count:', len(tweets_collector.get_scrapped_tweets()))

## 2. Scrap users

### Example 2.1.

Scrap users by usernames. Save them into memry, CSV and JSON lines.

In [ ]:
usernames = ['ProtasiewiczJ', 'donaldtuskEPP']
task = st.GetUsersTask(usernames)

user_collector = st.CollectorUserOutput()
outputs = [
    st.CsvUserOutput('users.csv'),
    st.JsonLineFileUserOutput('users.jl'),
    user_collector
]

task_result = st.GetUsersRunner(task, outputs).run()
[it.screen_name for it in user_collector.get_scrapped_users()]

## 3. Export data

### Example 3.1.

Export previous scrapped tweets into CSV and JSON line file.

In [ ]:
tweets = tweets_collector.get_scrapped_tweets()
st.export_tweets_to_csv(tweets, 'export_tweets.csv')
st.export_tweets_to_json_lines(tweets, 'export_tweets.jl')

### Example 3.2.

Export previous scrapped users into CSV and JSON line file.

In [ ]:
users = user_collector.get_scrapped_users()
st.export_users_to_csv(users, 'export_users.csv')
st.export_users_to_json_lines(users, 'export_users.jl')

## 4. Import data

### Example 4.1.

Import tweets from JSON lines file.

In [ ]:
tweets = st.read_tweets_from_json_lines_file('export_tweets.jl')

### Example 4.2.

Import tweets from CSV file.

In [ ]:
tweets = st.read_tweets_from_csv_file('export_tweets.csv')

### Example 4.3.

Import users from JSON lines file.

In [ ]:
users = st.read_users_from_json_lines_file('export_users.jl')

### Example 4.4.

Import users from CSV file.

In [ ]:
users = st.read_users_from_csv_file('export_users.csv')